In [1]:
from datascience import *
from datascience.predicates import are
path_data = '../../../../data/'
import numpy as np
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

from urllib.request import urlopen 
import re

from bancor_research.bancor_simulator.v3.spec.network import BancorDapp

# Recall from an earlier chapter that we already defined the whitelisted_tokens as follows.
whitelisted_tokens = {
    "eth": {
        "trading_fee": 0.01,
        "bnt_funding_limit": 1000000
    },
    "link": {
        "trading_fee": 0.01,
        "bnt_funding_limit": 1000000
    },
    "wbtc": {
        "trading_fee": 0.01,
        "bnt_funding_limit": 1000000
    }
}

# There are other possible configuration settings available, however for the present purpose we will use the defaults.
v3 = BancorDapp(whitelisted_tokens=whitelisted_tokens)

v3.set_user_balance(tkn_amt='101', tkn_name='eth', user_name='Alice')
v3.set_user_balance(tkn_amt='101', tkn_name='wbtc', user_name='Bob')
v3.set_user_balance(tkn_amt='10001', tkn_name='link', user_name='Charlie')
v3.set_user_balance(tkn_amt='23', tkn_name='bnt', user_name='Trader')
v3.set_user_balance(tkn_amt='1', tkn_name='eth', user_name='Trader')
v3.set_user_balance(tkn_amt='3', tkn_name='link', user_name='Trader')

# The actual deposits.

v3.deposit(tkn_amt='100', tkn_name='eth', user_name='Alice')

v3.deposit(tkn_amt='10000', tkn_name='link', user_name='Charlie')

v3.deposit(tkn_amt='100', tkn_name='wbtc', user_name='Bob')

v3.deposit(tkn_amt='1', tkn_name='eth', user_name='Alice')

v3.deposit(tkn_amt='1', tkn_name='link', user_name='Charlie')

v3.deposit(tkn_amt='1', tkn_name='wbtc', user_name='Bob')

for tkn_name in whitelisted_tokens:
    v3.enable_trading(tkn_name)

def read_url(url): 
    return re.sub('\\s+', ' ', urlopen(url).read().decode())

Bootstrap requirements met for eth
Bootstrap requirements met for link
Bootstrap requirements met for wbtc


# The Moving Average - EMA

A moving average is utilized as a security measure, where sudden changes in the pool reserves can be detected, and prevent abuse of the protocol's features. The moving average (ema) is updated with the first trade of the block, for any asset according to the following formula: 

![](https://lh5.googleusercontent.com/Hq0YWEHsrIza40yun3z0tuEu8migMiB6CzhIKZsY5TA2XCLw-zYqHGsm0J3QdB4-O1jxG1CF3swVjRyU4PTfQzyMJKxTCZRqS_D-9CbQYvcaZ-hh8_X2R_NGUqxIHigFGrpDeat7)

where r is the spot rate in units of BNT/TKN as determined by the trading liquidity balances of the pool, and α is an arbitrary constant that determines the responsiveness of the moving average. The α term is a global variable, set at 0.2 (or 20%) at launch of Bancor 3, and is intended to provide a consensus rate for the pool that is resistant to virtual price manipulation attacks. The following chart is an arbitrary depiction of the ema behavior relative to the spot price on a per-block basis. The ema is measured and updated before an action is executed; therefore, the ema response is delayed by a minimum of one action (e.g. a trade or add/remove liquidity event). Further, the ema is only adjusted once per pool, per block.


In [2]:
v3.describe(decimals=6)

eth          link          wbtc  \
1 Account Alice                         0.000000           NaN           NaN   
          Bob                                NaN           NaN      0.000000   
          Charlie                            NaN      0.000000           NaN   
          Trader                        1.000000      3.000000           NaN   
2 Pool    a: TKN Staked Balance       101.000000  10001.000000    101.000000   
          b: TKN Trading Liquidity     20.000000   3333.333333      1.250000   
          c: BNT Trading Liquidity  20000.000000  20000.000000  20000.000000   
          d: BNT Current Funding    20000.000000  20000.000000  20000.000000   
          e: Spot Rate               1000.000000      6.000000  16000.000000   
          f: Average Rate            1000.000000      6.000000  16000.000000   
          g: Average Inverse Rate       0.000000      0.000000      0.000000   
3 Network Master Vault                101.000000  10001.000000    101.000000   
          Protection Vault              0.000000      0.000000      0.000000   
          Protocol Equity                    NaN           NaN           NaN   
          Rewards Vault                      NaN           NaN           NaN   

                                          bnt       bneth        bnlink  \
1 Account Alice                           NaN  101.000000           NaN   
          Bob                             NaN         NaN           NaN   
          Charlie                         NaN         NaN  10001.000000   
          Trader                    23.000000         NaN           NaN   
2 Pool    a: TKN Staked Balance           NaN    0.000000      0.000000   
          b: TKN Trading Liquidity        NaN    0.000000      0.000000   
          c: BNT Trading Liquidity        NaN    0.000000      0.000000   
          d: BNT Current Funding          NaN    0.000000      0.000000   
          e: Spot Rate                    NaN    0.000000      0.000000   
          f: Average Rate                 NaN    0.000000      0.000000   
          g: Average Inverse Rate         NaN    0.000000      0.000000   
3 Network Master Vault                    NaN         NaN           NaN   
          Protection Vault                NaN         NaN           NaN   
          Protocol Equity                 NaN    0.000000      0.000000   
          Rewards Vault                   NaN    0.000000      0.000000   

                                        bnwbtc  
1 Account Alice                            NaN  
          Bob                       101.000000  
          Charlie                          NaN  
          Trader                           NaN  
2 Pool    a: TKN Staked Balance       0.000000  
          b: TKN Trading Liquidity    0.000000  
          c: BNT Trading Liquidity    0.000000  
          d: BNT Current Funding      0.000000  
          e: Spot Rate                0.000000  
          f: Average Rate             0.000000  
          g: Average Inverse Rate     0.000000  
3 Network Master Vault                     NaN  
          Protection Vault                 NaN  
          Protocol Equity             0.000000  
          Rewards Vault               0.000000

At genesis, the ema rate is set equal to the spot rate. Therefore, in the above scenario each of the liquidity pools began with the following rates:

In [3]:
v3.describe(decimals=4)

eth        link        wbtc  \
1 Account Alice                         0.0000         NaN         NaN   
          Bob                              NaN         NaN      0.0000   
          Charlie                          NaN      0.0000         NaN   
          Trader                        1.0000      3.0000         NaN   
2 Pool    a: TKN Staked Balance       101.0000  10001.0000    101.0000   
          b: TKN Trading Liquidity     20.0000   3333.3333      1.2500   
          c: BNT Trading Liquidity  20000.0000  20000.0000  20000.0000   
          d: BNT Current Funding    20000.0000  20000.0000  20000.0000   
          e: Spot Rate               1000.0000      6.0000  16000.0000   
          f: Average Rate            1000.0000      6.0000  16000.0000   
          g: Average Inverse Rate       0.0000      0.0000      0.0000   
3 Network Master Vault                101.0000  10001.0000    101.0000   
          Protection Vault              0.0000      0.0000      0.0000   
          Protocol Equity                  NaN         NaN         NaN   
          Rewards Vault                    NaN         NaN         NaN   

                                        bnt     bneth      bnlink    bnwbtc  
1 Account Alice                         NaN  101.0000         NaN       NaN  
          Bob                           NaN       NaN         NaN  101.0000  
          Charlie                       NaN       NaN  10001.0000       NaN  
          Trader                    23.0000       NaN         NaN       NaN  
2 Pool    a: TKN Staked Balance         NaN    0.0000      0.0000    0.0000  
          b: TKN Trading Liquidity      NaN    0.0000      0.0000    0.0000  
          c: BNT Trading Liquidity      NaN    0.0000      0.0000    0.0000  
          d: BNT Current Funding        NaN    0.0000      0.0000    0.0000  
          e: Spot Rate                  NaN    0.0000      0.0000    0.0000  
          f: Average Rate               NaN    0.0000      0.0000    0.0000  
          g: Average Inverse Rate       NaN    0.0000      0.0000    0.0000  
3 Network Master Vault                  NaN       NaN         NaN       NaN  
          Protection Vault              NaN       NaN         NaN       NaN  
          Protocol Equity               NaN    0.0000      0.0000    0.0000  
          Rewards Vault                 NaN    0.0000      0.0000    0.0000

However, the trading situations do have an effect. To demonstrate, assume that the trades described above happen on consecutive blocks. For the LINK trading pool, both the spot rate and the ema begin at 6, as set at the genesis of the pool. Since the ema is adjusted before the trade is executed, the first trade has no effect on the ema; the spot price changes as expected. The new spot price becomes relevant in the next block, as the ema is updated prior to performing the second trade. First, the ema is updated using the new spot rate, then the second trade is processed. In this example, the lag of the ema means there is a significant gap between it and the spot price after the first block; however, the adjustment in the second block, prior to executing the second trade, results in a close agreement thereafter.

In [4]:
timestep = v3.global_state.timestamp

v3.trade(tkn_amt='2000', source_token='bnt',  target_token='link', user_name='Trader', timestamp=timestep)

v3.describe(decimals=4)

eth        link        wbtc  \
1 Account Alice                         0.0000         NaN         NaN   
          Bob                              NaN         NaN      0.0000   
          Charlie                          NaN      0.0000         NaN   
          Trader                        1.0000    303.0000         NaN   
2 Pool    a: TKN Staked Balance       101.0000  10003.4242    101.0000   
          b: TKN Trading Liquidity     20.0000   3033.3333      1.2500   
          c: BNT Trading Liquidity  20000.0000  21995.6044  20000.0000   
          d: BNT Current Funding    20000.0000  20000.0000  20000.0000   
          e: Spot Rate               1000.0000      7.2513  16000.0000   
          f: Average Rate            1000.0000      6.0000  16000.0000   
          g: Average Inverse Rate       0.0000      0.0000      0.0000   
3 Network Master Vault                101.0000   9701.0000    101.0000   
          Protection Vault              0.0000      0.0000      0.0000   
          Protocol Equity                  NaN         NaN         NaN   
          Rewards Vault                    NaN         NaN         NaN   

                                           bnt     bneth      bnlink    bnwbtc  
1 Account Alice                            NaN  101.0000         NaN       NaN  
          Bob                              NaN       NaN         NaN  101.0000  
          Charlie                          NaN       NaN  10001.0000       NaN  
          Trader                    -1977.0000       NaN         NaN       NaN  
2 Pool    a: TKN Staked Balance            NaN    0.0000      0.0000    0.0000  
          b: TKN Trading Liquidity         NaN    0.0000      0.0000    0.0000  
          c: BNT Trading Liquidity         NaN    0.0000      0.0000    0.0000  
          d: BNT Current Funding           NaN    0.0000      0.0000    0.0000  
          e: Spot Rate                     NaN    0.0000      0.0000    0.0000  
          f: Average Rate                  NaN    0.0000      0.0000    0.0000  
          g: Average Inverse Rate          NaN    0.0000      0.0000    0.0000  
3 Network Master Vault                     NaN       NaN         NaN       NaN  
          Protection Vault                 NaN       NaN         NaN       NaN  
          Protocol Equity                  NaN    0.0000      0.0000    0.0000  
          Rewards Vault                    NaN    0.0000      0.0000    0.0000

In [5]:
v3.trade(tkn_amt='302.9981', source_token='link',  target_token='bnt', user_name='Trader', timestamp=timestep + 1)
v3.describe(decimals=4)

eth        link        wbtc  \
1 Account Alice                         0.0000         NaN         NaN   
          Bob                              NaN         NaN      0.0000   
          Charlie                          NaN      0.0000         NaN   
          Trader                        1.0000      0.0019         NaN   
2 Pool    a: TKN Staked Balance       101.0000  10003.4242    101.0000   
          b: TKN Trading Liquidity     20.0000   3336.3314      1.2500   
          c: BNT Trading Liquidity  20000.0000  20013.9939  20000.0000   
          d: BNT Current Funding    20000.0000  20015.9807  20000.0000   
          e: Spot Rate               1000.0000      5.9988  16000.0000   
          f: Average Rate            1000.0000      6.2503  16000.0000   
          g: Average Inverse Rate       0.0000      0.0000      0.0000   
3 Network Master Vault                101.0000  10003.9981    101.0000   
          Protection Vault              0.0000      0.0000      0.0000   
          Protocol Equity                  NaN         NaN         NaN   
          Rewards Vault                    NaN         NaN         NaN   

                                       bnt     bneth      bnlink    bnwbtc  
1 Account Alice                        NaN  101.0000         NaN       NaN  
          Bob                          NaN       NaN         NaN  101.0000  
          Charlie                      NaN       NaN  10001.0000       NaN  
          Trader                    0.6153       NaN         NaN       NaN  
2 Pool    a: TKN Staked Balance        NaN    0.0000      0.0000    0.0000  
          b: TKN Trading Liquidity     NaN    0.0000      0.0000    0.0000  
          c: BNT Trading Liquidity     NaN    0.0000      0.0000    0.0000  
          d: BNT Current Funding       NaN    0.0000      0.0000    0.0000  
          e: Spot Rate                 NaN    0.0000      0.0000    0.0000  
          f: Average Rate              NaN    0.0000      0.0000    0.0000  
          g: Average Inverse Rate      NaN    0.0000      0.0000    0.0000  
3 Network Master Vault                 NaN       NaN         NaN       NaN  
          Protection Vault             NaN       NaN         NaN       NaN  
          Protocol Equity              NaN    0.0000      0.0000    0.0000  
          Rewards Vault                NaN    0.0000      0.0000    0.0000

In [6]:
v3.trade(tkn_amt='1', source_token='eth',  target_token='wbtc', user_name='Trader', timestamp=timestep)
v3.describe(decimals=4)

eth        link        wbtc  \
1 Account Alice                         0.0000         NaN         NaN   
          Bob                              NaN         NaN      0.0000   
          Charlie                          NaN      0.0000         NaN   
          Trader                        0.0000      0.0019      0.0557   
2 Pool    a: TKN Staked Balance       101.0000  10003.4242    101.0005   
          b: TKN Trading Liquidity     21.0000   3336.3314      1.1943   
          c: BNT Trading Liquidity  19055.2381  20013.9939  20940.8835   
          d: BNT Current Funding    20007.6190  20015.9807  20000.0000   
          e: Spot Rate                907.3923      5.9988  17534.2111   
          f: Average Rate            1000.0000      6.2503  16000.0000   
          g: Average Inverse Rate       0.0000      0.0000      0.0000   
3 Network Master Vault                102.0000  10003.9981    100.9443   
          Protection Vault              0.0000      0.0000      0.0000   
          Protocol Equity                  NaN         NaN         NaN   
          Rewards Vault                    NaN         NaN         NaN   

                                       bnt     bneth      bnlink    bnwbtc  
1 Account Alice                        NaN  101.0000         NaN       NaN  
          Bob                          NaN       NaN         NaN  101.0000  
          Charlie                      NaN       NaN  10001.0000       NaN  
          Trader                    0.6153       NaN         NaN       NaN  
2 Pool    a: TKN Staked Balance        NaN    0.0000      0.0000    0.0000  
          b: TKN Trading Liquidity     NaN    0.0000      0.0000    0.0000  
          c: BNT Trading Liquidity     NaN    0.0000      0.0000    0.0000  
          d: BNT Current Funding       NaN    0.0000      0.0000    0.0000  
          e: Spot Rate                 NaN    0.0000      0.0000    0.0000  
          f: Average Rate              NaN    0.0000      0.0000    0.0000  
          g: Average Inverse Rate      NaN    0.0000      0.0000    0.0000  
3 Network Master Vault                 NaN       NaN         NaN       NaN  
          Protection Vault             NaN       NaN         NaN       NaN  
          Protocol Equity              NaN    0.0000      0.0000    0.0000  
          Rewards Vault                NaN    0.0000      0.0000    0.0000